In [13]:
from math import floor
mouthday={1:31,2:31,3:28,4:31,5:30,6:31,7:30,8:31,9:31,10:30,11:31,12:30}
def solvetimediff(t2, t1):
    t1sec = t1 % 100
    t2sec = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1min = t1 % 100
    t2min = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1hour = t1 % 100
    t2hour = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1day = t1 % 100
    t2day = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1mth = t1 % 100
    t2mth = t2 % 100
    if t2sec < t1sec:
        t2min = t2min - 1
        t3sec = t2sec + 60 - t1sec
    else:
        t3sec = t2sec - t1sec
    if t2min < t1min:
        t2hour = t2hour - 1
        t3min = t2min + 60 - t1min
    else:
        t3min = t2min - t1min
    if t2hour < t1hour:
        t2day = t2day - 1
        t3hour = t2hour + 24 - t1hour
    else:
        t3hour = t2hour - t1hour
    if t2day < t1day:
        t2mth = t2mth - 1
        t3day = t2day + mouthday[t2mth] - t1day
    else:
        t3day = t2day - t1day
    return t3sec+t3min*60+t3hour*3600+t3day*86400
#############查看日期的范围-->都是18年的  1 4 7 11月  所以上面的函数够用了
for i in range(5):
    pass
    #print(pd.read_csv(clear_train_data_withoutCharging_path+'\\'+clear_train_data_csvname[number]).time.describe())

In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import matplotlib.pylab as plt
%matplotlib inline
#输出格式控制   显示数据时不要将某栏目的数据缩略， 有多少数据显示多少数据
pd.set_option('display.max_colwidth', 100000)

#注意为了防止\造成的转义，在字符串前面加r以保持字符原始值  或者双斜杠
rare_data_path=r"C:\Users\14020\Desktop\NCBDC 2019"
rare_data_excel=r"train_dataset.xlsx"
clear_train_data_path=r"C:\Users\14020\Desktop\NCBDC 2019\Clear_TrainData"
clear_train_data_withoutCharging_path=r"C:\Users\14020\Desktop\NCBDC 2019\Clear_TrainData_WithoutCharging"#拿到分割正确的无充电状态表
clear_train_data_csvname=["Vehicle No.0.csv","Vehicle No.1.csv","Vehicle No.2.csv","Vehicle No.3.csv","Vehicle No.4.csv"]
def getvalidata(car,fragement):
    return pd.read_csv(r"C:\Users\14020\Desktop\NCBDC 2019\TestData\Vehicle No."+str(car)+"_Fragment No."+str(fragement)+".csv")

In [ ]:
#用来增加secdiff
def addsecdiff(df):
    timediff = []
    time = df.time.tolist()
    for index, t in enumerate(time):
        if(t==time[0]):
            timediff.append(0)
            continue
        timediff.append(solvetimediff(t,time[index-1]))
    #print(timediff)
    df['secdiff']=pd.Series(timediff)
    return df
dflist=[]


In [ ]:
for i in range(5):
    car0data=pd.read_csv(clear_train_data_withoutCharging_path+'\\'+clear_train_data_csvname[i])
    #发现最后一列有一个 Unnamed: 10 去除
    car0data=car0data.drop(['Unnamed: 10'],axis=1)
    #求出里程增量
    car0data['milediff']=car0data['mileage'].diff()
    #求出速度增量
    car0data['speeddiff']=car0data['speed'].diff()
    #求出电池P输出
    car0data['batteryP']=car0data['total_voltage']*car0data['total_current']
    #求出电机P输出
    car0data['motoP']=car0data['motor_voltage']*car0data['motor_current']
    #简单求出错误的时间差来筛选
    car0data['timediff']=car0data['time'].diff()

    # 分段
    segementTime=86400
    cutdata=car0data.loc[car0data['timediff']>segementTime].loc[car0data.milediff>1]
    cutpoint=cutdata.index.tolist()
    print(cutpoint)
    #得到所有连续段的左右端点list
    section=[]
    for index,i in enumerate(cutpoint):
        if index==0:
            section.append([0,cutpoint[0]-1])
        else:
            section.append([cutpoint[index-1],i-1])
        if i==cutpoint[-1]:
            section.append([i,len(car0data)-1])
    #筛选出中间数据大于10条的片段
    timediff=[]
    for i in section:
        if i[1]-i[0]>10:
            timediff.append(i)
    print(timediff)
    #连续片段第一条数据的 milediff speeddiff timediff设为0 
    for cutindex in timediff:
        car0data.loc[cutindex[0],'milediff']=0
        car0data.loc[cutindex[0],'speeddiff']=0
        car0data.loc[cutindex[0],'timediff']=0

    #给出连续时间片段
    for t in timediff:
        another_timediff = []
        time = car0data.time.tolist()
        for index, another_t in enumerate(time):
            if(index==0):
                another_timediff.append(0)
                continue
            another_timediff.append(solvetimediff(another_t,time[index-1]))
        car0data['secdiff']=another_timediff
        dflist.append(car0data.loc[t[0]:t[1],:])

In [25]:
#用来增加secdiff
def addsecdiff(df):
    timediff = []
    time = df.time.tolist()
    for index, t in enumerate(time):
        if(t==time[0]):
            timediff.append(0)
            continue
        timediff.append(solvetimediff(t,time[index-1]))
    print(timediff)
    df['secdiff']=pd.Series(timediff)
    return df
dflist=[]


In [49]:
df=addsecdiff(getvalidata(1,2))
print(len(df))
pd.set_option('display.max_colwidth', 100000)
df

[0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 

,time,total_voltage,total_current,soc,temp_max,temp_min,motor_voltage,motor_current,mileage,secdiff
0,20180225062053,539.0,28.0,100,17.0,11.0,542.0,6.0,40844.8,0
1,20180225062103,538.3,30.0,100,17.0,11.0,540.0,23.0,NaN,10
2,20180225062113,537.7,25.0,100,17.0,11.0,541.0,0.0,NaN,10
3,20180225062123,537.1,28.0,100,17.0,11.0,541.0,0.0,NaN,10
4,20180225062133,537.3,13.0,100,17.0,11.0,541.0,0.0,NaN,10
5,20180225062143,536.5,13.0,100,17.0,11.0,540.0,0.0,NaN,10
6,20180225062153,536.7,12.0,100,17.0,11.0,541.0,0.0,NaN,10
7,20180225062203,536.7,11.0,100,17.0,11.0,540.0,0.0,NaN,10
8,20180225062213,536.6,11.0,99,17.0,11.0,540.0,0.0,NaN,10
9,20180225062223,536.5,11.0,99,17.0,11.0,540.0,0.0,NaN,10


In [4]:
len(dflist)

31

In [5]:
alldata=pd.DataFrame()
for df in dflist:
    alldata=pd.concat([alldata,df])
alldata

,time,speed,total_voltage,total_current,soc,temp_max,temp_min,motor_voltage,motor_current,mileage,milediff,speeddiff,batteryP,motoP,timediff,secdiff
0,20180101064535,17.0,545.7,2.0,100.0,21.0,14.0,546.0,0.0,33891.8,0.0,0.0,1091.4,0.0,0.0,0
1,20180101064545,23.0,537.1,121.0,100.0,21.0,14.0,536.0,118.0,33891.8,0.0,6.0,64989.1,63248.0,10.0,10
2,20180101064555,36.0,535.1,57.0,99.0,21.0,14.0,535.0,53.0,33891.9,0.1,13.0,30500.7,28355.0,10.0,10
3,20180101064605,32.0,537.0,2.0,99.0,21.0,14.0,537.0,0.0,33892.0,0.1,-4.0,1074.0,0.0,50.0,10
4,20180101064615,1.0,537.6,3.0,99.0,21.0,14.0,538.0,0.0,33892.0,0.0,-31.0,1612.8,0.0,10.0,10
5,20180101064625,5.0,537.3,21.0,99.0,21.0,14.0,535.0,55.0,33892.0,0.0,4.0,11283.3,29425.0,10.0,10
6,20180101064635,33.0,531.2,78.0,99.0,21.0,14.0,531.0,81.0,33892.1,0.1,28.0,41433.6,43011.0,10.0,10
7,20180101064645,37.0,532.5,38.0,99.0,21.0,14.0,532.0,38.0,33892.2,0.1,4.0,20235.0,20216.0,10.0,10
8,20180101064655,33.0,534.9,2.0,99.0,21.0,14.0,535.0,8.0,33892.3,0.1,-4.0,1069.8,4280.0,10.0,10
9,20180101064705,35.0,532.5,43.0,99.0,21.0,14.0,533.0,15.0,33892.4,0.1,2.0,22897.5,7995.0,50.0,10



# 处理好了分段问题，现在对每一段的训练数据，我们求其中的secdiff
如果不分段，某些中间偷偷跑的距离就会被学习（然而这在测试数据中是没有的）

In [6]:
len(dflist)

31

In [7]:
input_attribute=[ 'secdiff','batteryP','motoP','total_voltage','total_current','soc','temp_max','temp_min','motor_voltage','motor_current']
output_attribute=['milediff']#'speeddiff',
input_shape=len(input_attribute)
output_shape=len(output_attribute)
import numpy as np
#手动归一化处理 因为之前是一个一个df加入dflist的 alldata是后面累加的，所以无法直接对alldata作用，而是用alldata数据对下面这个数据作用
for index,df in enumerate(dflist):
    data=df.copy()
    mycolumns=data.columns
    data.total_voltage=(data.total_voltage-alldata.total_voltage.min())/(alldata.total_voltage.max()-alldata.total_voltage.min()) #min max
    #data.total_current=(data.total_current-alldata.total_current.mean())/alldata.total_current.std()#不可以 会失去反向电压特征
    data.total_current=data.total_current/alldata.total_current.std()
    data.motor_voltage=(data.motor_voltage-alldata.motor_voltage.min())/(alldata.motor_voltage.max()-alldata.motor_voltage.min()) #min max
    #data.motor_current=(data.motor_current-alldata.motor_current.mean())/alldata.motor_current.std()#不用改
    data.motor_current=data.motor_current/alldata.motor_current.std()
    data.soc=data.soc/100
    data.secdiff=data.secdiff/10
    data.temp_max=(data.temp_max-alldata.temp_max.min())/(alldata.temp_max.max()-alldata.temp_max.min()) #min max
    data.temp_min=(data.temp_min-alldata.temp_min.min())/(alldata.temp_min.max()-alldata.temp_min.min()) #min max
    data.batteryP=(data.batteryP-alldata.batteryP.mean())/alldata.batteryP.std()#Standardization
    data.motoP=(data.motoP-alldata.motoP.mean())/alldata.motoP.std()#Standardization
    string = r"C:\Users\14020\Desktop\NCBDC 2019\Normalized_data_list"+'\\'+str(index)
    np.save(string,data)
    

In [12]:
dflist[0].columns

Index(['time', 'speed', 'total_voltage', 'total_current', 'soc', 'temp_max',
       'temp_min', 'motor_voltage', 'motor_current', 'mileage', 'milediff',
       'speeddiff', 'batteryP', 'motoP', 'timediff', 'secdiff'],
      dtype='object')

In [9]:
path=r"C:\Users\14020\Desktop\NCBDC 2019\Normalized_data_list"+'\\'
np.load(path+'0.npy') 

array([[  2.01801011e+13,   1.70000000e+01,   6.02923264e-01, ...,
         -3.19391096e-01,   0.00000000e+00,   0.00000000e+00],
       [  2.01801011e+13,   2.30000000e+01,   4.98172960e-01, ...,
          2.53033073e+00,   1.00000000e+01,   1.00000000e+00],
       [  2.01801011e+13,   3.60000000e+01,   4.73812424e-01, ...,
          9.58180722e-01,   1.00000000e+01,   1.00000000e+00],
       ..., 
       [  2.01801131e+13,   1.00000000e+00,   3.97076736e-01, ...,
         -3.19391096e-01,   1.00000000e+01,   1.00000000e+00],
       [  2.01801131e+13,   0.00000000e+00,   3.98294762e-01, ...,
         -3.19391096e-01,   1.00000000e+01,   1.00000000e+00],
       [  2.01801131e+13,   0.00000000e+00,   3.98294762e-01, ...,
         -3.19391096e-01,   1.00000000e+01,   1.00000000e+00]])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.layers import Activation
from keras.layers.recurrent import SimpleRNN
from keras.utils.vis_utils import plot_model
import numpy as np
from keras.layers import  Dropout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import random
import sys
np.random.seed(7)

#https://blog.csdn.net/clayanddev/article/details/53947012#rnn-using-keras
class RNNKeras:
    
    def __init__(self, time_steps, vector_size, output_size, hidden_dim=100,dropout=0.1,units=50):
        # Assign instance variables
        self.time_steps = time_steps
        self.vector_size = vector_size
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.dropout=dropout
        self.units=units

        self.__model_build__()
    
    def __model_build__(self):
        self.model = Sequential()

        self.model.add(LSTM(self.output_size, input_shape=(self.time_steps, self.vector_size)))
        self.model.add(Dense(self.vector_size))
        self.model.add(Activation('sigmoid'))
        
        #optimizer = RMSprop(lr=0.01)
        self.model.compile(loss='mse', optimizer='adam')
    
    def train_model(self, X, y, batchSize=128, nepoch=10):
        self.model.fit(X, y, batch_size=batchSize, epochs=nepoch,validation_split=0)

    def predict(self, x):
        return model.predict(x, verbose=0)[0]

In [12]:
from math import ceil
TimeStep=10 #相当于其他地方的sequence_length 用多少前面的数据去预测最后一个
model=RNNKeras(time_steps=TimeStep,vector_size=input_shape,output_size=output_shape,hidden_dim=10)
#准备数据
######################################
data=data[:25000]
X=data.loc[:,input_attribute]
Y=data.loc[:,output_attribute]
X=X.to_numpy()
Y=Y.to_numpy()
trainX,trainY=[],[]#暂时存储所有输入给rnn的数据 等等分训练集和测试集
datasize=len(X)
trainsize=ceil(datasize/10*8)
testsize=datasize-trainsize
print("数据量",datasize,"训练数据",trainsize,"测试数据",testsize)
for i in range(datasize-TimeStep-1):
    x=X[i:i+TimeStep]
    y=Y[i:i+TimeStep].sum()#此处可能需要改，改成 里层增量的和
    trainX.append(x.tolist())
    trainY.append(y)
#得到了所有正确格式的输入和输出 trianX的格式需要是（batch_size, timestep，featurenum
#分训练集和测试集
########################ctrl+/ 注释多行
# testX=np.array(trainX[trainsize:])
# testY=np.array(trainY[trainsize:])
# trainX=np.array(trainX[:trainsize])
# trainY=np.array(trainY[:trainsize])
testX=trainX[trainsize:]
testY=trainY[trainsize:]
trainX=trainX[:trainsize]
trainY=trainY[:trainsize]

数据量 25000 训练数据 20000 测试数据 5000


In [25]:
np.array(trainY).shape

(20000,)

In [21]:
np.array(trainX).shape

(20000, 10, 10)

In [26]:
model.train_model(trainX,trainY)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 20000 arrays: [array([[ 0.        , -0.34444867, -0.3193911 ,  0.60292326,  0.04053166,
         1.        ,  0.45238095,  0.375     ,  0.94463668,  0.        ],
       [ 1.        ,  2.10941442,  2.53033073,  0.49...

In [96]:
import tensorflow_datasets as tfds
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,as_supervised=True)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


...

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Dataset imdb_reviews downloaded and prepared to C:\Users\14020\tensorflow_datasets\imdb_reviews\subwords8k\0.1.0. Subsequent calls will reuse this data.


In [ ]:
train_dataset, test_dataset = dataset['train'], dataset['test']
tokenizer = info.features['text'].encoder
print('vocabulary size: ', tokenizer.vocab_size)
